In [1]:
from pathlib import Path
import csv

In [2]:
p = Path(r'D:\python\some_tools\dxf_test\skb20')
cgtow = p / 'cgtow.txt'
tabs = []   # таблицы по каждой фазе

запишем лист с координатами опор из файла

In [3]:
ctw = []
with cgtow.open() as f:
    spam = csv.reader(f, delimiter='\t', skipinitialspace=True)
    for row in spam:
        if not row[1] == '':
            ctw.append(row)
ctw[1][1]

'468188.6'

In [4]:
dxf_files = list(p.glob('001*.dxf'))   # list of dxf files

создаем функцию парсер dxf

In [5]:
def dxf_poly_parse(path):
    
    dxf = []   # list to write all data from file
    
    with open(path) as dxf_file:
        for line in dxf_file:
            dxf.append(line.strip())   # write each stroke to list
            
    polylines = []   # empty list for polylines
    line = []   # list for coords of each polyline

    for i in range(len(dxf)):
        if dxf[i] == 'POLYLINE':
            line = []   # list cleaning
        if dxf[i] == 'VERTEX' and dxf[i+1] == '10':
            line.append((float(dxf[i+2]), float(dxf[i+4]), float(dxf[i+6])))
        if dxf[i] == 'SEQEND':
            polylines.append(line)
    
    return polylines

делаем функцию рассчета расстояния 2д

In [6]:
def dist(x1, y1, x2, y2):
    """ расчет расстояния """
    return (((x2-x1)**2 + (y2-y1)**2)**1/2)

проход по дхф файлам

In [76]:
mid_points = []   # list for mid points
for file in dxf_files:
    mids = []   # list of mid points for each phase
    phase = file.stem.split('_')[-1]   # name of phase
    polylines = dxf_poly_parse(file)   # calling function to have polylines
    
    for line in range(len(polylines)):
        d1_min = d2_min = 1000000    # fake distance
        id_start = id_end = 'None'   # fake id
 
        st_x = polylines[line][0][0]
        st_y = polylines[line][0][1]
        end_x = polylines[line][-1][0]
        end_y = polylines[line][-1][1]
        
        # finding start and end ID
        for a in range(len(ctw)):
            # calc if one of other structures is closer to the start of polyline
            
            d2 = dist(st_x, st_y, float(ctw[a][1]), float(ctw[a][2]))
            
            if d2 < d1_min:
                d1_min = d2   # rewrite min dist
                id_start = ctw[a][0]   # rewrite id
                
        for b in range(len(ctw)):
            # calc if one of other structures is closer to the end of polyline
            d2 = dist(end_x, end_y, float(ctw[b][1]), float(ctw[b][2]))
            
            if d2 < d2_min:
                d2_min = d2   # rewrite min dist
                id_end = ctw[b][0]   # rewrite id
        
        mid_x = st_x - (st_x-end_x)/2
        mid_y = st_y - (st_y-end_y)/2
        
        # finding mid_z
        cv_num = len(polylines[line])//2   # central vertex number
        
        d_cv_mid = dist(polylines[line][cv_num][0], polylines[line][cv_num][1], mid_x, mid_y)
        d_b_vert = dist(polylines[line][cv_num][0], polylines[line][cv_num][1], polylines[line][cv_num+1][0], polylines[line][cv_num+1][1])
        mid_z = polylines[line][cv_num][2] - ((polylines[line][cv_num][2] - polylines[line][cv_num+1][2])*(d_cv_mid/d_b_vert))
        
        mids.append((phase, id_start, id_end, round(mid_x, 2),  round(mid_y, 2), round(mid_z, 2)))
    mid_points.append(mids)


In [28]:
import pandas as pd

In [77]:
mp = {}
for tab in mid_points:
    mp[tab[0][0]] = pd.DataFrame(tab, columns=['phase', 'from', 'to', 'x', 'y', 'z'])

In [79]:
mp['1a']

,phase,from,to,x,y,z
0,1a,ПС Суоярви,1,468162.95,6885564.24,168.81
1,1a,1,2,468249.77,6885579.90,168.82
2,1a,2,3,468370.10,6885628.12,169.06
3,1a,3,4,468502.47,6885691.98,165.93
4,1a,4,5,468642.39,6885759.24,167.06
...,...,...,...,...,...,...
152,1a,149,150,482591.47,6907680.27,160.22
153,1a,150,151,482717.25,6907770.70,163.67
154,1a,151,152,482815.48,6907839.93,167.19
155,1a,152,153,482859.54,6907911.69,163.15


In [ ]:
# del(mid_points)

In [87]:
for key in mp.keys():
    mp[key]['span'] = mp[key][['from', 'to']].agg(' - '.join, axis=1)

In [91]:
mp_ab = mp['1a'].merge(mp['1b'], left_on='span', right_on='span', suffixes=('_1a', '_1b'))

In [96]:
mp_ab = mp_ab.drop(['phase_1a', 'phase_1b', 'from_1b', 'to_1b'], axis=1)

KeyError: "['phase_1a' 'phase_1b' 'from_1b' 'to_1b'] not found in axis"